In [64]:
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [46]:
# configure gmaps to use your Google API key
gmaps.configure(api_key=g_key)

In [71]:
# import city weather data
city_data_df = pd.read_csv("Data/WeatherPy_challenge.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,Alice Springs,AU,-23.70,133.88,41.0,80,10,3.36,clear sky,0,0
1,Punta Arenas,CL,-53.15,-70.92,42.8,93,90,11.41,mist,0,0
2,Klaksvik,FO,62.23,-6.59,46.4,93,100,13.87,drizzle,0,0
3,Puerto Ayora,EC,-0.74,-90.35,80.6,65,40,14.99,scattered clouds,0,0
4,Los Llanos De Aridane,ES,28.66,-17.92,77.0,78,75,11.41,broken clouds,0,0


In [48]:
# ask customer for temperature, rain, & snow preferences
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain = input("Do you want it to be raining? (yes/no) ")
snow = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [49]:
# filter the dataset to find the cities that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (inches)"] == 0) & \
                                       (city_data_df["Snow (inches)"] == 0)]
preferred_cities_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
3,Puerto Ayora,EC,-0.74,-90.35,80.60,65,40,14.99,scattered clouds,0,0
4,Los Llanos De Aridane,ES,28.66,-17.92,77.00,78,75,11.41,broken clouds,0,0
5,Hithadhoo,MV,-0.60,73.08,84.06,71,96,6.26,overcast clouds,0,0
6,Kalangala,UG,-0.31,32.23,71.60,83,20,9.17,few clouds,0,0
8,Albany,US,42.60,-73.97,78.80,24,40,10.29,scattered clouds,0,0


In [50]:
# get city count
preferred_cities_df.count()

City                   249
Country                249
Lat                    249
Lng                    249
Max Temp               249
Humidity               249
Cloudiness             249
Wind Speed             249
Current Description    249
Rain (inches)          249
Snow (inches)          249
dtype: int64

In [51]:
# create df called hotel_df to store hotel names, city, country, current weather description, & max temp
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Puerto Ayora,EC,80.60,scattered clouds,-0.74,-90.35,
4,Los Llanos De Aridane,ES,77.00,broken clouds,28.66,-17.92,
5,Hithadhoo,MV,84.06,overcast clouds,-0.60,73.08,
6,Kalangala,UG,71.60,few clouds,-0.31,32.23,
8,Albany,US,78.80,scattered clouds,42.60,-73.97,


In [52]:
# parameters to search for a hotel (radius = 5000 m)
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [54]:
# iterate through hotel_df
for index, row in hotel_df.iterrows():
    # get latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]

    # add latitude and longitude to location key for params dictionary
    params["location"] = f"{lat},{lng}"

    # use the search term: "lodging" and our latitude and longitude
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # make request and get the JSON data from the search
    hotels = requests.get(base_url, params=params).json()
    # grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [57]:
# preview hotel_df
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Puerto Ayora,EC,80.60,scattered clouds,-0.74,-90.35,Finch Bay Galapagos Hotel
4,Los Llanos De Aridane,ES,77.00,broken clouds,28.66,-17.92,Valle Aridane
5,Hithadhoo,MV,84.06,overcast clouds,-0.60,73.08,Scoop Guest House
6,Kalangala,UG,71.60,few clouds,-0.31,32.23,Ssese Habitat Resort
8,Albany,US,78.80,scattered clouds,42.60,-73.97,


In [72]:
# save hotel_df as .csv
hotel_df.to_csv("Data/WeatherPy_vacation.csv", index=False)

In [69]:
# info box template for map markers
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# store hotel_df row in info box
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# add a heatmap of temperature for the vacation spots and marker for each city
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))